<a href="https://colab.research.google.com/github/gabisaitop/Clusters/blob/main/AtividadeSwift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tkinter import font
import seaborn as sns
import datetime

In [ ]:
df = pd.read_csv('Swift_SALES.csv')
df.info()

FileNotFoundError: ignored

In [ ]:
df_list = df[['id_cliente', 'id_pedido', 'item_id', 'preco', 'horario_pedido']]
df.head()

In [ ]:
df_list.head()

In [ ]:
df_compra = df.groupby('id_cliente').horario_pedido.max().reset_index()
df_compra.columns = ['id_cliente', 'DataMaxCompra']
df_compra.head()

In [ ]:
df_compra['DataMaxCompra'] = pd.to_datetime(df_compra['DataMaxCompra'])

In [ ]:
df_compra['Recencia'] = (datetime.datetime.today() - df_compra['DataMaxCompra']).dt.days
df_compra.head()

In [ ]:
df_cliente = pd.merge(df_list, df_compra[['id_cliente', 'Recencia']], on = 'id_cliente')
df_cliente.head()

In [ ]:
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt

df_recencia = df_cliente[['Recencia']]

wcss=[]
for i in range(2, 10):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=len(df_recencia), n_init=10, random_state=0)
    kmeans.fit(df_recencia)
    wcss.append(kmeans.inertia_)

plt.plot(range(2,10), wcss)
plt.title('Metodo do cotovelo')
plt.show()

In [ ]:
import math

def numero_otimo_cluster(wcss, cluster_final) :
  x0, y0 = 2, wcss[0]
  x1, y1 = cluster_final, wcss[-1]
  distancia = []

  for i in range(len(wcss)):
    x = i + 2
    y = wcss[i]

    numerator = abs((y1 - y0)*x - (x1 - x0)*y + x1*y0 - y1*x0)
    denominator = math.sqrt((y1-y0)**2 + (x1-x0)**2)
    distance=(numerator/denominator)
    distancia.append(distance)
    
  return distancia.index(max(distancia))+2

print('O cluster ideal é: ', numero_otimo_cluster(wcss, 10))

In [ ]:
KMeans = KMeans(n_clusters=4)
df_cliente['RecenciaCluster'] = KMeans.fit_predict(df_recencia)
df_cliente.head()

In [ ]:
#calcular a media de cada cluster
agrupado_por_cluster = df_cliente.groupby('RecenciaCluster')['Recencia'].mean().reset_index()
agrupado_por_cluster.head()

In [ ]:
#colocar em ordem para dar pontuação dos clientes... quando autor a media da recencia... mais pontos...
agrupado_por_cluster_ordenado = agrupado_por_cluster.sort_values(by='Recencia', ascending=False).reset_index()
agrupado_por_cluster_ordenado.head()

In [ ]:
#add uma nova coluna "index score" para salvar a ordenacao
agrupado_por_cluster_ordenado['score'] =  agrupado_por_cluster_ordenado.index
agrupado_por_cluster_ordenado = agrupado_por_cluster_ordenado.drop(['index'], axis = 1)
agrupado_por_cluster_ordenado.head()

In [ ]:
#Pereparar para adicionar[
juntando_cluster = pd.merge(df_cliente, agrupado_por_cluster_ordenado [['RecenciaCluster', 'score']], on='RecenciaCluster')
juntando_cluster.head()

In [ ]:
#removendo os dados da coliuna cluster
removendo_dados = juntando_cluster.drop(['RecenciaCluster'], axis=1)
removendo_dados.head()

In [ ]:
df_final = removendo_dados.rename(columns={'score': 'RecenciaScore'})
df_final.head()

In [ ]:
#função gráfica
def plot_perc(st, dados):
  g = sns.countplot(x=st, data=dados, orient='h')
  g.set_ylabel('Valores')
  sizes = []

  for p in g.patches:
    height = p.get_height()
    sizes.append(height)
    g.text(p.get_x() + p.get_width()/1.6, height + 200, '{:1.2f}%'.format(height/116581*100), ha='center', va='bottom', fontSize=12)

  g.set_ylim(0, max(sizes)* 1.1)

In [ ]:
plot_perc('RecenciaScore', df_final)

In [ ]:
#Sugestão: montar funcao para adicionar e organizar o SCORE por cluster
# - cluster_name: campo com os grupos de cluster
# - target_name: campo objeto de analise
# - score_name: nome final do campo após finalizar o processo
# - df: DataFrame com o resultado do K-means
# - ordem: True - crescente / False - decrescente

def ordernador_cluster(cluster_name, target_name, score_name, df, ordem):
    agrupado_por_cluster = df.groupby(cluster_name)[target_name].mean().reset_index()
    agrupado_por_cluster_ordenado = agrupado_por_cluster.sort_values(by=target_name,ascending=ordem).reset_index(drop=True)
    agrupado_por_cluster_ordenado['score'] = agrupado_por_cluster_ordenado.index
    juntanto_cluster = pd.merge(df, agrupado_por_cluster_ordenado[[cluster_name,'score']], on=cluster_name)
    removendo_dados = juntanto_cluster.drop([cluster_name],axis = 1)
    df_final = removendo_dados.rename(columns={'score': score_name})
    return df_final


In [ ]:
#carregar na base de cliente
df_cliente = ordernador_cluster('RecenciaCluster','Recencia','RecenciaScore', df_cliente, False)
df_cliente.head()

In [ ]:
df_final.groupby('RecenciaScore')['Recencia'].describe()

In [ ]:
#mostra estrutura da base de dados
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116581 entries, 0 to 116580
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   amostra               116581 non-null  int64  
 1   id_cliente            116581 non-null  object 
 2   id_unico_cliente      116581 non-null  object 
 3   item_id               116581 non-null  int64  
 4   cep_cliente           116581 non-null  int64  
 5   cidade_cliente        116581 non-null  object 
 6   estado_cliente        116581 non-null  object 
 7   id_pedido             116581 non-null  object 
 8   status_pedido         116581 non-null  object 
 9   horario_pedido        116581 non-null  object 
 10  pedido_aprovado       116567 non-null  object 
 11  pedido_entregador     115368 non-null  object 
 12  pedido_entregue       114066 non-null  object 
 13  data_estimada         116581 non-null  object 
 14  id_vendedor           116581 non-null  object 
 15  

In [ ]:
#calcular o score para frequencia:
#definir campo de frequencia
df_frequencia = df.groupby('id_cliente').pedido_aprovado.count().reset_index()
df_frequencia.columns = ['id_cliente', 'Frequencia']
print(df_frequencia)

                             id_cliente  Frequencia
0      00012a2ce6f8dcda20d059ce98491703           1
1      000161a058600d5901f007fab4c27140           1
2      0001fd6190edaaf884bcaf3d49edf079           1
3      0002414f95344307404f0ace7a26f1d5           1
4      000379cdec625522490c315e70c7a9fb           1
...                                 ...         ...
97250  fffcb937e9dd47a13f05ecb8290f4d3e           1
97251  fffecc9f79fd8c764f843e9951b11341           3
97252  fffeda5b6d849fbd39689bb92087f431           1
97253  ffff42319e9b2d713724ae527742af25           1
97254  ffffa3172527f765de70084a7e53aae8           2

[97255 rows x 2 columns]


In [ ]:
#2o etapa:
#adicionar o calculo no df_cliente:
df_cliente = pd.merge(df_cliente, df_frequencia, on='id_cliente') #precisa saber qual o campo de ligação p fazer a conexao
print(df_cliente)

                              id_cliente                         id_pedido  \
0       06b8999e2fba1a1fbc88172c00ba8bc7  00e7ee1b050b8499577073aeb2a297a1   
1       f0ac8e5a239118859b1734e1087cbb1f  b1a5d5365d330d10485e0203d54ab9e8   
2       fd3a0b1bd209f0e7d420c9c3d1127613  574fe1739f65af76badd0999db300b4f   
3       fbd40c083aa8cddebb5265b2ba6aaf2e  e0b26f14d2bcc710bb02f77a4628763b   
4       10558ef4afea173bfb5e2cbe3d5b0bb5  eaae5bd20fb15d85aa673d9b7c0e8ca5   
...                                  ...                               ...   
116576  ce2172509c4149d65212484eb761bc37  edcf1e1eeb52381be9388c90152be52d   
116577  ee1cfdc92e449920e25d3ca4ab4da4f6  2ef4a11b6e24fdfbb43b92cb5f95edff   
116578  ee1cfdc92e449920e25d3ca4ab4da4f6  2ef4a11b6e24fdfbb43b92cb5f95edff   
116579  ab0cf72dfe0538a63a57d6905ccb7b57  2c12150c742ae2fa48bc703964c16c5f   
116580  c61336d6c835632048f973d2f2db598d  9f7ba492c273ffac8f90a623b8eadd3b   

        item_id   preco       horario_pedido  Recencia  Recenci

In [ ]:
df_cliente.head()

,id_cliente,id_pedido,item_id,preco,horario_pedido,Recencia,RecenciaScore,Frequencia
0,06b8999e2fba1a1fbc88172c00ba8bc7,00e7ee1b050b8499577073aeb2a297a1,1,124.99,2020-05-16 15:05:35,907,0,1
1,f0ac8e5a239118859b1734e1087cbb1f,b1a5d5365d330d10485e0203d54ab9e8,1,124.99,2020-05-07 20:11:26,915,0,1
2,fd3a0b1bd209f0e7d420c9c3d1127613,574fe1739f65af76badd0999db300b4f,1,126.99,2020-03-23 15:10:17,961,0,1
3,fbd40c083aa8cddebb5265b2ba6aaf2e,e0b26f14d2bcc710bb02f77a4628763b,1,124.99,2020-05-16 10:00:49,907,0,1
4,10558ef4afea173bfb5e2cbe3d5b0bb5,eaae5bd20fb15d85aa673d9b7c0e8ca5,1,126.99,2020-03-18 23:04:36,965,0,1


In [ ]:
#criar um df só com frequencia, pois estamos mexendo com apenas um eixo
#definindo os clusters (grupos):
df_frequencia = df_cliente[['Frequencia']]
kmeans = KMeans(n_clusters=4)
df_cliente['FrequenciaCluster'] = kmeans.fit_predict(df_frequencia) #cria uma coluna nova chama frequencia cluster dentro de df_cliente
df_cliente.head()


TypeError: ignored